In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class NeuropixelSimulator:
    def __init__(self, num_channels=2000, motifs=None):
        """
        Initialize the NeuropixelSimulator.
        
        :param num_channels: Number of recording channels (default is 2000)
        :param motifs: List of motifs (each motif is an object)
        """
        self.num_channels = num_channels
        self.motifs = motifs if motifs is not None else []

    def simulate_online(self, num_samples=1000000, num_motifs=5, motif_repeats=10, noise_level=0.01):
        """
        Simulate 'online' mode where a defined number of motifs are present in the data.
    
        :param num_samples: Number of samples to simulate
        :param num_motifs: Number of different motifs to include
        :param motif_repeats: Number of times each motif occurs in the data
        :param noise_level: Amount of noise to add to the data
        :return: A numpy array (num_channels x num_samples) with simulated data
        """
        # Initialize the data array with random noise (random firing)
        data = np.random.normal(0, noise_level, (self.num_channels, num_samples))
    
        # Embed the motifs in the data
        for i in range(num_motifs):
            motif = self.motifs[i % len(self.motifs)]
            motif_channels = motif.shape[0]  # Number of channels in the motif
            motif_length = motif.shape[1]    # Length of the motif (in samples)
    
            for _ in range(motif_repeats):
                start_idx = np.random.randint(0, num_samples - motif_length)
                
                # Randomly select a subset of channels to place the motif
                channel_indices = np.random.choice(self.num_channels, motif_channels, replace=False)
                
                # Add the motif to the selected channels and sample range
                data[channel_indices, start_idx:start_idx + motif_length] += motif
    
        return data


    def simulate_offline(self, num_samples=1000000, motif_distribution_factor=0.5, noise_level=0.01):
        """
        Simulate 'offline' mode where motifs are replayed but distributed differently.

        :param num_samples: Number of samples to simulate
        :param motif_distribution_factor: How much to distribute the motif across the network (0 to 1)
        :param noise_level: Amount of noise to add to the data
        :return: A numpy array (num_channels x num_samples) with simulated data
        """
        # Initialize the data array with random noise
        data = np.random.normal(0, noise_level, (self.num_channels, num_samples))

        # Replay motifs with different distributions
        for motif in self.motifs:
            motif_repeats = np.random.randint(5, 15)
            for _ in range(motif_repeats):
                start_idx = np.random.randint(0, num_samples - motif.shape[1])
                distributed_motif = self.distribute_motif(motif, motif_distribution_factor)
                data[:, start_idx:start_idx + distributed_motif.shape[1]] += distributed_motif

        return data

    def distribute_motif(self, motif, factor):
        """
        Distribute a motif across more channels based on the distribution factor.
        
        :param motif: The original motif
        :param factor: How much to distribute the motif across channels (0 = none, 1 = fully spread)
        :return: A modified motif distributed across more channels
        """
        distributed_motif = np.zeros((self.num_channels, motif.shape[1]))

        num_motif_channels = motif.shape[0]
        spread_channels = int(num_motif_channels + (self.num_channels - num_motif_channels) * factor)
        
        channels_idx = np.random.choice(self.num_channels, spread_channels, replace=False)
        distributed_motif[channels_idx[:num_motif_channels], :] = motif

        return distributed_motif

    def plot_simulation(self, data, num_channels_to_plot=50, sample_range=(0, 1000)):
        """
        Plot the simulated data, mimicking neural recordings.

        :param data: The 2D numpy array (num_channels x num_samples) to plot
        :param num_channels_to_plot: Number of channels to display
        :param sample_range: Range of samples to plot (start, end)
        """
        plt.figure(figsize=(10, 6))
        channels_to_plot = np.random.choice(self.num_channels, num_channels_to_plot, replace=False)
        for i, ch in enumerate(channels_to_plot):
            plt.plot(data[ch, sample_range[0]:sample_range[1]] + i*5, color='black', lw=0.5)
        plt.xlabel('Time (Samples)')
        plt.ylabel('Channels')
        plt.title(f'Simulated Neural Recording ({num_channels_to_plot} channels)')
        plt.show()


# Example Usage
if __name__ == "__main__":
    # Create dummy motifs for testing
    motif_1 = np.zeros((10, 100))  # A motif across 10 channels and 100 time points
    motif_2 = np.zeros((15, 150))  # Another motif across 15 channels and 150 time points
    
    # Add random spikes in the motifs (just for testing)
    motif_1[3, 20:30] = 1
    motif_1[6, 40:50] = 1
    motif_2[2, 50:70] = 1
    motif_2[8, 90:110] = 1

    # Initialize the simulator with the motifs
    simulator = NeuropixelSimulator(motifs=[motif_1, motif_2])

    # Simulate 'online' mode
    online_data = simulator.simulate_online(num_samples=10000, num_motifs=2, motif_repeats=5, noise_level=0.02)

    # Simulate 'offline' mode
    offline_data = simulator.simulate_offline(num_samples=10000, motif_distribution_factor=0.7, noise_level=0.02)

    # Plot the online simulation result
    simulator.plot_simulation(online_data, num_channels_to_plot=30, sample_range=(0, 1000))

    # Plot the offline simulation result
    simulator.plot_simulation(offline_data, num_channels_to_plot=30, sample_range=(0, 1000))
